In [1]:
!pip install selenium
!apt-get update # for installing ChromeDriver
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 23.1 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,722 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get

In [13]:
## 자기 번호 수정하고 가세요 ~~~!~!~!~!~!~!~!~!~!~!
START, END = 20, 30  # 각자 해야할 파트
save_base_path = '/content/drive/MyDrive/AirbnbDataset/c2_2_20-29'  # 여기에 저장할 거야

# ※ 건들지 마시오.
SLEEP = 2
ROOM_LIST = ['주방', '거실', '침실']

In [14]:
import os, time, re, json
from bs4 import BeautifulSoup
import requests
from requests import request
from requests.compat import urljoin
import base64
from urllib.parse import quote, unquote, urlencode, urljoin
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from google.colab import drive

# sys.path 설정
import sys

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
# 도시 리스트
# cities = ['New York', 'Los Angeles', 'San Francisco', 'Las Vegas', 'Miami', 'Chicago', 'Orlando', 'Vancouver',
#           'Toronto', 'Montreal', 'Quebec City', 'London', 'Edinburgh', 'Manchester', 'Liverpool', 'Paris', 'Nice', 'Lyon', 'Marseille',
#           'Berlin', 'Munich', 'Hamburg', 'Frankfurt', 'Rome', 'Venice', 'Milan', 'Florence', 'Naples', 'Barcelona', 'Seville', 'Madrid', 'Valencia',
#           'Sydney', 'Melbourne', 'Brisbane', 'Perth', 'Tokyo', 'Osaka', 'Kyoto', 'Fukuoka', 'Sapporo', 'Seoul', 'Busan', 'Jeju', 'Gangneung', 'Beijing',
#           'Shanghai', 'Guangzhou', 'Chengdu', 'Bangkok', 'Phuket', 'Chiang Mai', 'Pattaya', 'Hanoi', 'Ho Chi Minh', 'Mexico City', 'Da Nang', 'Cancun',
#           'Guadalajara', 'Rio de Janeiro', 'São Paulo', 'Istanbul', 'Cappadocia', 'Antalya']

cities = ["Amsterdam", "Prague", "Vienna", "Budapest", "Copenhagen", "Stockholm", "Oslo", "Helsinki", "Dublin",
    "Reykjavik", "Athens", "Santorini", "Dubrovnik", "Split", "Krakow", "Warsaw", "Bruges", "Brussels", "Zurich", "Geneva",
    "Dubai", "Abu Dhabi", "Doha", "Jerusalem", "Tel Aviv", "Cairo", "Marrakesh", "Cape Town", "Johannesburg", "Nairobi",
    "Buenos Aires", "Lima", "Cusco", "Santiago", "Bogotá", "Cartagena", "Quito", "Panama City", "San José",
    "Havana", "Punta Cana", "San Juan", "Kingston", "Manila", "Bali", "Jakarta", "Kuala Lumpur", "Singapore", "Taipei"]
cities = [it.replace(' ', '-') for it in cities]
Curl = list()
for i in range(len(cities)):
    Curl.append(f'https://www.airbnb.co.kr/s/{cities[i]}/homes')

In [16]:
# 이미지 저장하는 함수
def save_images(img_list, category, room_dir):
    category_dir = os.path.join(room_dir, category)
    os.makedirs(category_dir, exist_ok=True)
    for idx_img, img_url in enumerate(img_list):
        try:
            img_data = requests.get(img_url, headers=headers).content
            img_filename = f"{category}_{idx_img}.jpg"
            img_path = os.path.join(category_dir, img_filename)
            with open(img_path, 'wb') as f:
                f.write(img_data)
            print(f"✅ 저장 완료: {img_path}")
        except Exception as e:
            print(f"❌ 이미지 다운로드 실패: {e}")

# 도시 페이지 바꿔주는 거
def make_cursor(page: int) -> str:
    items_offset = (page - 1) * 18
    obj = {"section_offset": 0, "items_offset": items_offset, "version": 1}
    json_str = json.dumps(obj, separators=(',', ':'))
    b64 = base64.b64encode(json_str.encode()).decode()
    return quote(b64)

In [17]:
# --- 6. 숙소별 이미지 크롤링 및 저장 함수 ---
def crawl_and_save_images(driver, urls, i):
    idx = 0
    while urls:
        url = urls.pop()
        try:
            print(f"▶{i}/{END-1}▶ {idx+1}/{270}: {url}")

            # modal url 생성
            if '?' in url:
                p1, p2 = re.split(r'\?', url)
                modal_url = p1 + '?modal=PHOTO_TOUR_SCROLLABLE&' + p2
            else:
                modal_url = url + '?modal=PHOTO_TOUR_SCROLLABLE'

            driver.get(modal_url)
            time.sleep(SLEEP)

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # 이미지 분류할 리스트
            rooms = dict()

            for img_tag in soup.find_all('img', alt=True):
                if 12 > len(img_tag['alt']) > 6 and any(rl in img_tag['alt'] for rl in ROOM_LIST):
                    rooms[img_tag['alt'][:-5]] = rooms.get(img_tag['alt'][:-5], set())
                    rooms[img_tag['alt'][:-5]].add(img_tag['src'])

            # 이미지 가치 판단
            for category, img_set in rooms.items():
                if len(img_set) >= 3:
                    # 숙소 고유번호 추출
                    room_number = url.split('/rooms/')[-1].split('?')[0].strip()
                    if room_number in pre_room_list:
                        print(f'⚠️ 숙소 크롤링한 숙소:{room_number}')
                        continue
                    room_dir = os.path.join(save_base_path, room_number)
                    os.makedirs(room_dir, exist_ok=True)

                    with open(f'/content/drive/MyDrive/AirbnbDataset/room_number.txt', 'a', encoding='utf-8') as file:
                        file.write(f'{room_number}\n')

                    save_images(list(img_set), category, room_dir)

        except Exception as e:
            print(f"⚠️ 숙소 처리 중 오류: {e}")
            continue

        idx += 1

In [18]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Accept": "*/*",
    "Accept-Language": "ko-KR,ko;q=0.9",
    "Referer": "https://www.airbnb.co.kr/",
    "Cookie": (
    "rclmd=%7B%22309469461%22%3A%22otp_phone%22%7D; "
    "rclu=%7B%22309469461%22%3A%2250vhLm912hLLTAL9pCkv7QgUL4zGAFn3aj5IMQ97e2A%3D%22%7D; "
    "hli=1; "
    "li=1; "
    "tzo=540")
}
os.makedirs(save_base_path, exist_ok=True)
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

# --- 4. 셀레니움 드라이버 설정 (Colab용) ---
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument('--window-size=1920x1080')
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
if __name__ == '__main__':
    urls = list()

    import time
    # --- 5. 도시별 숙소 링크 수집 함수 ---
    for i in range(START, END):
        current_time = time.time()


        pre_room_list = list()
        pre_room_list_path = '/content/drive/MyDrive/AirbnbDataset/room_number.txt'
        with open(pre_room_list_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            lines = [line.strip() for line in lines]

        pre_room_list.extend(lines)
        pre_room_list = set(pre_room_list)

        curl = Curl[i] # 첫번째 뉴욕 -> ...

        cpgs = list() # 뉴욕이면 뉴욕에 대한 페이지들
        for j in range(1, 16): # 15페이지까지 있으니깐
            cursor = make_cursor(j)
            cpgs.append(f"{curl}?cursor={cursor}")

        for urll in cpgs:
            resp = request('get',urll)
            dom = BeautifulSoup(resp.text, 'html.parser')

            for a_tag in dom.select('a[href]'):
                if re.search(r'^/rooms/', a_tag['href']):
                    link = urljoin('https://www.airbnb.co.kr', a_tag['href'])
                    if link not in urls:
                        urls.append(link) # 나랑 똑같애야됨

        crawl_and_save_images(driver, urls, i)
        print(f"\n✅ {curl} 모든 작업 완료!\t소요 시간 : {time.time() - current_time}")

Streaming output truncated to the last 5000 lines.
✅ 저장 완료: /content/drive/MyDrive/AirbnbDataset/c2_2_20-29/1416081037705081826/거실/거실_0.jpg
✅ 저장 완료: /content/drive/MyDrive/AirbnbDataset/c2_2_20-29/1416081037705081826/거실/거실_1.jpg
✅ 저장 완료: /content/drive/MyDrive/AirbnbDataset/c2_2_20-29/1416081037705081826/거실/거실_2.jpg
✅ 저장 완료: /content/drive/MyDrive/AirbnbDataset/c2_2_20-29/1416081037705081826/거실/거실_3.jpg
✅ 저장 완료: /content/drive/MyDrive/AirbnbDataset/c2_2_20-29/1416081037705081826/거실/거실_4.jpg
▶21/29▶ 214/270: https://www.airbnb.co.kr/rooms/1388933482748721021?search_mode=regular_search&adults=1&check_in=2025-06-10&check_out=2025-06-15&children=0&infants=0&pets=0&source_impression_id=p3_1747138279_P30mUOaoD9vu3veA&previous_page_section_name=1000&federated_search_id=66881016-26fd-4904-845c-db96373f6b7b
▶21/29▶ 215/270: https://www.airbnb.co.kr/rooms/49284140?search_mode=regular_search&adults=1&check_in=2025-05-26&check_out=2025-05-31&children=0&infants=0&pets=0&source_impression_id=p3_1747

# 데이터 확인용 파트

In [ ]:
!pip install matplotlib pillow

import os, time
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display, clear_output
import shutil
import warnings; warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
def load_rooms_number(root):
    room_list = list()
    for name in ['del.txt', 'save.txt']:
        try:
            with open(os.path.join(root, name), 'r', encoding='utf-8') as f:
                lines = f.readlines()
                lines = [line.strip() for line in lines]
            room_list.extend(lines)
        except:
            pass
    return set(room_list)

def save_rooms_number(save_root, file_name, room_number):
    with open(f'{save_root}/{file_name}.txt', 'a', encoding='utf-8') as file:
        file.write(f'{room_number}\n')

def show_select_images(imgs_path, rn):
    imgs = os.listdir(imgs_path)

    ## show images

    _, axes = plt.subplots(2, 4, figsize=(10, 6))
    axes = axes.ravel()
    imgs.sort()
    print(f'방 번호 : {rn}')

    for i, img_name in enumerate(imgs):
        img_path = os.path.join(imgs_path, img_name)
        img = Image.open(img_path)
        axes[i].imshow(img)
        axes[i].set_title(img_name)
        axes[i].axis('off')

    for j in range(i + 1, len(axes)):
        axes[j].axis('off')
    plt.tight_layout()
    plt.show()
    time.sleep(0.1)
    clear_output(wait=True)

    ## select imagess

    user_input = map(int, input("이미지 번호 입력: ").split())
    user_input = list(set(filter(lambda n: 0 <= n < len(imgs), user_input)))

    # save images and

    if len(user_input) > 2:
        save_rooms_number(save_root,'save', rn)

        save_path = os.path.join(save_root, rn)
        url_room_num = 0

        while os.path.exists(save_path):
            save_path += '_' + str(url_room_num)
        os.makedirs(save_path, exist_ok=True)

        for i, num in enumerate(user_input):
            copy_path = os.path.join(imgs_path, imgs[num])
            save_image = os.path.join(save_path, f'{i}.jpg')
            shutil.copy(copy_path, save_image)
    else:
        save_rooms_number(save_root, 'del', rn)

In [ ]:
if __name__ == '__main__':
    ## 경로 설정
    root = '/content/drive/MyDrive/AirbnbDataset/Images'

    save_root = '/content/drive/MyDrive/AirbnbDataset/airbnb_images'
    os.makedirs(save_root, exist_ok=True)

    pre_room_number = load_rooms_number(save_root)

    for it in sorted(os.listdir(root)):
        for rn in os.listdir(os.path.join(root, it)):
            if rn not in pre_room_number:
                for acc in os.listdir(os.path.join(root, it, rn)):
                    show_select_images(os.path.join(root, it, rn, acc), rn)

NameError: name 'load_rooms_number' is not defined

In [ ]:
##

In [ ]:
import os

def save_rooms_number(save_root, file_name, room_number):
    with open(f'{save_root}/{file_name}.txt', 'a', encoding='utf-8') as file:
        for it in room_number:
            file.write(f'{it}\n')

root = '/content/drive/MyDrive/AirbnbDataset/airbnb_images'

for it in os.listdir(root):
    path = os.path.join(root, it)
    rns = list(); rnss = set()
    for rn in os.listdir(path):
       rns.append(rn); rnss.add(rn)
    print(it, len(rns), len(rnss))

airbnb_img_30-39 514 514
airbnb_img_0-9 950 950
airbnb_img_28-29 106 106
airbnb_img_40-49 950 950
airbnb_img_50-58 1063 1063
airbnb_img_60-64 323 323
.ipynb_checkpoints 0 0


NotADirectoryError: [Errno 20] Not a directory: '/content/drive/MyDrive/AirbnbDataset/Images/room_number.txt'

In [12]:
import os
r = 0
a = '/content/drive/MyDrive/AirbnbDataset/c2_2_10-19'
for it in os.listdir(a):
    r += 1

print(r)

498


In [ ]:
'''
1-r : 5059
2-r : 3770 ->  74.52%
3-r : 2484 -> 49.1%
4-r : 2368 -> 46.8%
'''

'''
1-r : 3786
2-r :
'''

3770


In [ ]:
522+498

3786